### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from demo.scenarios.session import *

### Helper Functions
General functions and external imports.

In [ ]:
# General functions.

from demo.scenarios import garden
import numpy as np
import pandas as pd
from scipy import stats
from os import path


def load_data(data_folder: str):
    """Loads all garden data results and taxonomy categories."""
    df_results = garden.load_base_results(data_folder, "predictions_test.csv")
    df_results.head()

    # Load the taxonomic data and merge with results.
    df_info = garden.load_taxonomy(data_folder)
    df_results.rename(columns={"label": "Label"}, inplace=True)
    df_all = garden.merge_taxonomy_with_results(df_results, df_info)

    return df_info, df_all


def load_results(data_folder: str):
    """loads reproducabilty test result runs"""
    # my_folder = data_folder +
    df_results = pd.read_csv(
        path.join(data_folder, "ReproducibilityDataSet_CV.csv")
    )

    return df_results
    # df_results = garden.load_base_results(data_folder,)

In [ ]:
# Prepare the data. For this section, instead of executing the model, we will use CSV files containing the results of an already executed run of the model.
# df_info, df_all = load_data(DATASETS_DIR)

# df = pd.read_csv('ReproducibilityDataSet_CV.csv')

df = load_results(DATASETS_DIR)

In [ ]:
results = stats.friedmanchisquare(
    df.Accuracy_r0,
    df.Accuracy_r1,
    df.Accuracy_r2,
    df.Accuracy_r3,
    df.Accuracy_r4,
    df.Accuracy_r5,
    df.Accuracy_r6,
    df.Accuracy_r7,
    df.Accuracy_r8,
    df.Accuracy_r9,
)
results.pvalue

### Measurements

In this first example, we simply wrap the output from `accuracy_score` with a custom `Result` type to cope with the output of a third-party library that is not supported by a MLTE builtin.

In [ ]:
from mlte.evidence.types.array import Array
from mlte.measurement.external_measurement import ExternalMeasurement


kruskal_measurement = ExternalMeasurement(
    "repeated training on training samples", Array, stats.friedmanchisquare
)

# Evaluate.
kruskal_res = kruskal_measurement.evaluate(
    df.Accuracy_r0,
    df.Accuracy_r1,
    df.Accuracy_r2,
    df.Accuracy_r3,
    df.Accuracy_r4,
    df.Accuracy_r5,
    df.Accuracy_r6,
    df.Accuracy_r7,
    df.Accuracy_r8,
    df.Accuracy_r9,
)

# Inspect values
print(kruskal_res)

# Save to artifact store
kruskal_res.save(force=True)